In [1]:
from model import MInterface
from data import DInterface


In [2]:
model = MInterface.load_from_checkpoint('checkpoints/egnn/2023-02-26-21-17-54-best-epoch=45-val_loss=0.04.ckpt')

In [3]:
data = DInterface(dataset='zinc_complex3a6p_data', data_dir='data/3a6p/zinc_drug_like_100k/3a6p_pocket5_202020', batch_size=128)

Processing...
100%|██████████| 100446/100446 [00:15<00:00, 6625.69it/s]
Done!


In [4]:
data.setup(stage='test')
for batch in data.test_dataloader():
    p, x = model.model.forward_(batch)
    break


In [19]:

molecule_pre = list()
molecule_end = list()
atoms = list()
for i in range(128):
    molecule_pre.append(batch.pos[batch.batch==i].detach().numpy())
    molecule_end.append(x[batch.batch==i].detach().numpy())
    atoms.append(batch.x[batch.batch==i].detach().numpy())

In [7]:
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.axes import Axes
import numpy as np
from torch import Tensor
matplotlib.use('Qt5Agg')


In [40]:
elements_dict = {0: 'C', 1: 'H', 2: 'N', 3: 'O', 4: 'F', 5: 'S', 6: 'CL', 7: 'BR', 8: 'I', 9: 'P'}
# elements_dict = {'C': 0, 'H': 1, 'N': 2, 'O': 3, 'F': 4, 'S': 5, 'CL': 6, 'BR': 7, 'I': 8, 'P': 9}
fig = plt.figure()
fig: plt.Figure
ax = fig.add_subplot(projection='3d')
ax: Axes
t = 0
ax.scatter(molecule_pre[t][:,0], molecule_pre[t][:,1], molecule_pre[t][:,2], c='b')
# 加入分子的id
molecules = batch.id.tolist()
ax.text2D(0.05, 0.95, "Molecule ID: ZINC{}".format(str(molecules[t]+int(1e12))[1:]), transform=ax.transAxes)
print("Molecule ID: ZINC{}".format(str(molecules[t]+int(1e12))[1:]))
ax.scatter(molecule_end[t][:,0], molecule_end[t][:,1], molecule_end[t][:,2], c='r')
for i in range(molecule_pre[t].shape[0]):
    ax.text(molecule_pre[t][i,0], molecule_pre[t][i,1], molecule_pre[t][i,2], elements_dict[atoms[t][i]], 'x')
for i in range(molecule_end[t].shape[0]):
    ax.plot([molecule_pre[t][i,0], molecule_end[t][i,0]], [molecule_pre[t][i,1], molecule_end[t][i,1]], [molecule_pre[t][i,2], molecule_end[t][i,2]], c='g')
# 将对应的两个点连起来
plt.show()

Molecule ID: ZINC001528757027


In [22]:
molecule_pre[t][:,0]
atoms[t]

array([2, 1, 0, 2, 1, 3, 2, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0, 0], dtype=int64)

In [32]:
%matplotlib

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
x = np.arange(0, 10, 0.1)
x = x.reshape(2, -1)
y = np.sin(x)
z = np.cos(x)
ax.plot(x, y)
plt.show()

Using matplotlib backend: Qt5Agg


ValueError: operands could not be broadcast together with remapped shapes [original->remapped]: (100,)  and requested shape (2,)